In [ ]:
import numpy as np
import json
from cad_viewer_widget import CadViewer, AnimationTrack
from sidecar import Sidecar

names = ["box", "box1", "boxes", "edges", "faces", "hexapod", "hexapod_m", "vertices"]
objects = {}
states = {}

for name in names:
    with open(f"examples/{name}.json", "r") as fd:
        objects[name] = json.load(fd)

    with open(f"examples/{name}-states.json", "r") as fd:
        states[name] = json.load(fd)

In [ ]:
sc = Sidecar(title="Tests")

with sc:
    cv = CadViewer(
        cad_width=640, 
        height=400, 
        tree_width=250, 
        theme="light", 
        tools=True
    )

In [ ]:
name = "hexapod"

cv.add_shapes(
    objects[name],
    states[name],
    ortho=True,
    control="trackball",
    ticks=20,
    axes=True,
    axes0=False,
    grid=[True, False, False], 
    transparent=False,
    black_edges=False,
    edge_color="#707070",
    ambient_intensity=0.5,
    direct_intensity=0.3,
    reset_camera=True,
#    position=(-523.1876106053778, 220.0057378967463, 786.3865378864517),
#    quaternion=(-0.3113879332023978, 0.011019379723961274, 0.8909337230080648, 0.33038345832295263),
#    zoom=0.5,
#    timeit=True,
#    animation_loop=False
)

In [ ]:
cv.js_debug = True

# Widget comms

In [ ]:
cv.update_states({
    '/bottom/bottom_0': (1,0),
    '/bottom/top/top_0': [0,1],
})

In [ ]:
cv.update_states({
    '/bottom/bottom_0': (1,1),
    '/bottom/top/top_0': [1,1],
})

In [ ]:
p0 = cv.position
if cv.control == "trackball":
    q0 = cv.quaternion
else:
    q0 = None
p0, q0

In [ ]:
if cv.control == "trackball":
    cv.set_camera(
        [-581.2983866303276,200.51522639763488,748.4089730676135],
        [-0.33190544711158626,0.026748479971868442,0.9128279093702769,0.23636518540500767])
else:
    cv.set_camera([-581.2983866303276,200.51522639763488,748.4089730676135])

cv.zoom = 0.5

In [ ]:
if cv.control == "trackball":
    cv.set_camera(p0, q0)
else:
    cv.set_camera(p0)
cv.zoom = 1

In [ ]:
cv.ambient_intensity = 0.9
cv.direct_intensity = 0.5

In [ ]:
cv.ambient_intensity = 0.5
cv.direct_intensity = 0.3

In [ ]:
ec = cv.edge_color

In [ ]:
cv.edge_color = "#ff0000"

In [ ]:
cv.edge_color = ec

In [ ]:
cv.grid = [not g for g in cv.widget.grid]

In [ ]:
cv.axes = not cv.axes
cv.axes0 = not cv.axes0
cv.transparent = not cv.transparent
cv.black_edges = not cv.black_edges

In [ ]:
cv.tools = not cv.tools

In [ ]:
cv.ortho = not cv.ortho

In [ ]:
cv.zoom_speed = 5
cv.pan_speed = 5
cv.rotate_speed = 5

In [ ]:
cv.zoom_speed =1
cv.pan_speed =1
cv.rotate_speed =1

In [ ]:
cv.last_pick

# Animation

In [ ]:
import numpy as np
horizontal_angle = 25

leg_names = {
    "right_back", "right_middle", "right_front", 
    "left_back",  "left_middle", "left_front",
    
}

def intervals(count):
    r = [ min(180, (90 + i*(360 // count)) % 360) for i in range(count)]
    return r 

def times(end, count):
    return np.linspace(0, end, count+1).tolist()
    
def vertical(count, end, offset, reverse):
    ints = intervals(count)
    heights = [round(35 * np.sin(np.deg2rad(x)) - 15, 1) for x in ints]
    heights.append(heights[0])
    return times(end, count), heights[offset:] + heights[1:offset+1]

def horizontal(end, reverse):
    factor = 1 if reverse else -1
    return times(end, 4), [0, factor * horizontal_angle, 0, -factor * horizontal_angle, 0]

leg_group = ("left_front", "right_middle", "left_back")

In [ ]:
tracks = []

for name in leg_names:
    # move upper leg
    cv.add_track(AnimationTrack(f"/bottom/{name}", "rz", *horizontal(4, "middle" in name)))

cv.animate(5)

In [ ]:
cv.stop()

In [ ]:
for name in leg_names:
    # move lower leg
    cv.add_track(AnimationTrack(f"/bottom/{name}/lower", "rz", *vertical(8, 4, 0 if name in leg_group else 4, "left" in name)))
    
cv.animate(2)

In [ ]:
cv.clear_tracks()

# Clipping handling

In [ ]:
cv.select_clipping()

In [ ]:
cv.clip_intersection = not cv.clip_intersection

In [ ]:
cv.clip_planes = not cv.clip_planes

In [ ]:
cv.clip_value_0 = 10
cv.clip_value_1 = -50
cv.clip_value_2 = 40

In [ ]:
cv.clip_normal_0

In [ ]:
cv.clip_value_2

In [ ]:
cv.clip_normal_0 = (-0.35, -0.35, -0.35)

In [ ]:
cv.clip_normal_0 = (-1, 0, 0)

In [ ]:
cv.select_tree()

# BACKUP

In [ ]:
import math

def rotate_x(vector, angle):
    angle = rad(angle)
    mat = np.array(
        [
            [1, 0, 0],
            [0, math.cos(angle), -math.sin(angle)],
            [0, math.sin(angle), math.cos(angle)],
        ]
    )
    return tuple(np.matmul(mat, vector))


def rad(deg):
    return deg / 180.0 * math.pi


def rotate_y(vector, angle):
    angle = rad(angle)
    mat = np.array(
        [
            [math.cos(angle), 0, math.sin(angle)],
            [0, 1, 0],
            [-math.sin(angle), 0, math.cos(angle)],
        ]
    )
    return tuple(np.matmul(mat, vector))


def rotate_z(vector, angle):
    angle = rad(angle)
    mat = np.array(
        [
            [math.cos(angle), -math.sin(angle), 0],
            [math.sin(angle), math.cos(angle), 0],
            [0, 0, 1],
        ]
    )
    return tuple(np.matmul(mat, vector))


def rotate(vector, angle_x=0, angle_y=0, angle_z=0):
    v = tuple(vector)
    if angle_z != 0:
        v = rotate_z(v, angle_z)
    if angle_y != 0:
        v = rotate_y(v, angle_y)
    if angle_x != 0:
        v = rotate_x(v, angle_x)
    return v